In [1]:
import os, sys

import sgpp
import dproc
import sgutil
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print(sys.version)

from sklearn.pipeline import make_pipeline

3.12.6 (main, Sep 30 2024, 02:19:13) [GCC 9.4.0]


- Host_Popularity_percentage 범주형 변수화
  
- Guest_Popularity_percentage 범주형 변수화

- Host_Popularity_percentage 를 -1 ~ 1 사이로 MinMaxScaling 

- Number_of_Ads에 이상치가 존재합니다. 이를 최빈값으로 정리하고 0 ~ 1 사이로 MinMaxScaling

- Guest_Popularity_percentage 를 -1 ~ 1 사이로 MinMaxScaling

- Episode_Length_minutes 는 범주형 변수화

- Episode_Length_minutes를 5에서 120 사이로

In [88]:
sc = sgutil.SGCache('img', 'result', 'model')

In [82]:
p = make_pipeline(
    sgpp.PolarsProcessor(),
    sgpp.ExprProcessor({
        'ELm_num': (pl.col('Episode_Length_minutes').clip(5, 120) - 5) / 115,
        'GP': pl.col('Guest_Popularity_percentage').clip(0, 100) / 50 ,
        'HP': pl.col('Host_Popularity_percentage').clip(0, 100) / 50,
        'NAd': (pl.when(pl.col('Number_of_Ads') > 4).then(0).otherwise(pl.col('Number_of_Ads'))).fill_null(0.0) /3 ,
        'Number_of_Ads': (pl.when(pl.col('Number_of_Ads') > 4).then(0).otherwise(pl.col('Number_of_Ads'))).fill_null(0).cast(pl.Int8),
        'ELm_na': pl.col('Episode_Length_minutes').is_null(),
        'GPp_na': pl.col('Guest_Popularity_percentage').is_null(),
        'ELm_sqrt': ((pl.col('Episode_Length_minutes').clip(5, 120) - 5) / 115)  ** 0.5
    }),
    sgpp.PandasConverter(index_col = 'id')
)
p.fit(['data/train.csv'])

Pipeline(steps=[('polarsprocessor',
                 <sgpp.PolarsProcessor object at 0x7fd531501e50>),
                ('exprprocessor',
                 <sgpp.ExprProcessor object at 0x7fd531502630>),
                ('pandasconverter',
                 <sgpp.PandasConverter object at 0x7fd5315024e0>)])

In [83]:
df_train = p.transform(['data/train.csv'])
df_train_1 = df_train.loc[df_train['Episode_Length_minutes'].notna()]
target = 'Listening_Time_minutes'

- Target Encoding 실험 재설계,
  > Episode_Length_minutes 제외한 모든 변수로 모든 데이터셋으로 Combination 1 ~ 4
  >
  > Episode_Length_minutes 포함 모든 변수로 Episode_Length_minutes 미결측 데이터셋으로 Combination 0 ~ 3

In [93]:
from itertools import combinations
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm 
from cuml.preprocessing import TargetEncoder

kf = KFold(4, shuffle = True, random_state = 123)
def get_tgt_rmse(df, X_tgt, smooth_space = np.linspace(1, 10, 10)):
    rmse_fold = list()
    best_prd = None
    best_rmse = np.inf
    for i in smooth_space:
        tgt = TargetEncoder(smooth = i, split_method ='continuous')
        prds = list()
        rmses = list()
        for train_idx, test_idx in kf.split(df[X_tgt], df[target]):
            df.iloc[train_idx].pipe(lambda x: tgt.fit(x[X_tgt], x[target]))
            df_valid = df.iloc[test_idx]
            prds.append(
                pd.Series(tgt.transform(df_valid[X_tgt]), index = df_valid.index)
            )
            rmses.append(
                root_mean_squared_error(df_valid[target], prds[-1])
            )
        rmse_fold.append(np.mean(rmses))
        if best_rmse > rmse_fold[-1]:
            best_rmse = rmse_fold[-1]
            best_prd = pd.concat(prds)
    return np.min(rmse_fold), smooth_space[np.argmin(rmse_fold)], best_prd

def get_tgt_rmse_list(df, X_values, smooth_space = [0.01, 0.1, 1, 10, 100, 1000]):
    results = list()
    best_prd_list = list()
    for i in tqdm(X_values):
        rmse, best_smooth, best_prd = get_tgt_rmse(df, i, smooth_space = smooth_space)
        results.append(
            pd.Series(
                [i, rmse, best_smooth], index = ['X_tgt', 'RMSE', 'smooth']
            )
        )
        best_prd_list.append(best_prd.rename('__'.join(i)))
    return pd.DataFrame(results).sort_values('RMSE'), pd.concat(best_prd_list, axis=1).sort_index()

In [96]:
X_val = [
    'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads',
    'Episode_Sentiment', 'Episode_Title', 'Genre', 'Podcast_Name', 'Publication_Day', 'Publication_Time'
]


# ALL

In [97]:
df_c1, df_prd_1 = sc.cache_result(
    'tgt_rmse_c1_all', lambda : get_tgt_rmse_list(df_train, [list(i) for i in combinations(X_val, 1)]), rerun = 0
)
df_c1

  0%|          | 0/9 [00:00<?, ?it/s]

,X_tgt,RMSE,smooth
0,[Host_Popularity_percentage],26.851036,10
2,[Number_of_Ads],26.923613,100
1,[Guest_Popularity_percentage],26.991587,100
4,[Episode_Title],27.064105,100
6,[Podcast_Name],27.104318,100
3,[Episode_Sentiment],27.117177,100
8,[Publication_Time],27.130197,1000
5,[Genre],27.131325,1000
7,[Publication_Day],27.134080,1000


In [98]:
df_c2, df_prd_2 = sc.cache_result(
    'tgt_rmse_c2_all', lambda : get_tgt_rmse_list(df_train, [list(i) for i in combinations(X_val, 2)]), rerun = 0
)
df_c2.iloc[:25]

  0%|          | 0/36 [00:00<?, ?it/s]

,X_tgt,RMSE,smooth
0,"[Host_Popularity_percentage, Guest_Popularity_...",26.614200,0.1
1,"[Host_Popularity_percentage, Number_of_Ads]",26.720847,10.0
3,"[Host_Popularity_percentage, Episode_Title]",26.815162,10.0
16,"[Number_of_Ads, Episode_Title]",26.848316,10.0
8,"[Guest_Popularity_percentage, Number_of_Ads]",26.882301,10.0
2,"[Host_Popularity_percentage, Episode_Sentiment]",26.884671,10.0
18,"[Number_of_Ads, Podcast_Name]",26.885627,10.0
15,"[Number_of_Ads, Episode_Sentiment]",26.904024,100.0
10,"[Guest_Popularity_percentage, Episode_Title]",26.909860,10.0
17,"[Number_of_Ads, Genre]",26.914936,100.0


In [99]:
df_c3, df_prd_3 = sc.cache_result(
    'tgt_rmse_c3_all', lambda : get_tgt_rmse_list(df_train, [list(i) for i in combinations(X_val, 3)]), rerun = 0
)
df_c3.iloc[:25]

  0%|          | 0/84 [00:00<?, ?it/s]

,X_tgt,RMSE,smooth
1,"[Host_Popularity_percentage, Guest_Popularity_...",26.624471,0.10
6,"[Host_Popularity_percentage, Guest_Popularity_...",26.636031,0.10
0,"[Host_Popularity_percentage, Guest_Popularity_...",26.639306,0.10
19,"[Host_Popularity_percentage, Episode_Title, Po...",26.654809,1.00
5,"[Host_Popularity_percentage, Guest_Popularity_...",26.683332,0.01
20,"[Host_Popularity_percentage, Episode_Title, Pu...",26.697376,1.00
41,"[Guest_Popularity_percentage, Episode_Title, P...",26.732151,1.00
40,"[Guest_Popularity_percentage, Episode_Title, P...",26.749837,0.10
7,"[Host_Popularity_percentage, Number_of_Ads, Ep...",26.762050,10.00
12,"[Host_Popularity_percentage, Number_of_Ads, Pu...",26.780514,10.00


In [ ]:
df_c4, df_prd_4 = sc.cache_result(
    'tgt_rmse_c4_all', lambda : get_tgt_rmse_list(df_train, [list(i) for i in combinations(X_val, 4)]), rerun = 0
)
df_c4.iloc[:25]

# Episode_Length_minutes Not NA

In [ ]:
df_c0_2, df_prd_0_2 = sc.cache_result(
    'tgt_rmse_c0_2', lambda : get_tgt_rmse_list(df_train_1, ['Episode_Length_minutes']), rerun = 0
)
df_c0_2

In [ ]:
df_c1_2, df_prd_1_2 = sc.cache_result(
    'tgt_rmse_c1_2', 
    lambda : get_tgt_rmse_list(df_train_1, [['Episode_Length_minutes'] + list(i) for i in combinations(X_val, 1)]), rerun = 0
)
df_c1_2

In [ ]:
df_c2_2, df_prd_2_2 = sc.cache_result(
    'tgt_rmse_c2_2', 
    lambda : get_tgt_rmse_list(df_train_1, [['Episode_Length_minutes'] + list(i) for i in combinations(X_val, 2)]), rerun = 0
)
df_c2_2